#### _OTECH-2685-create-and-update-public.encr_

https://hintmd.atlassian.net/browse/OTECH-2685

#### _part 1 : write python utility to load data governance csv into s3_

In [165]:
_pocnumber = '0.0.2'

In [2]:
_jiralink = 'https://hintmd.atlassian.net/browse/OTECH-2685'

In [48]:
import sys
import json
import os 
import logging
import xlrd
import pandas as pd
import numpy as np
import platform

In [4]:
from typing import cast, Sequence, Optional, Any, Dict, List, Union
from datetime import datetime
from pprint import pprint

In [5]:
logging.basicConfig(stream=sys.stdout,level=logging.INFO)
logger = logging.getLogger(__name__)
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.propagate = False

In [6]:
_today = datetime.today()
_today

datetime.datetime(2021, 12, 1, 12, 35, 37, 364578)

In [7]:
logger.info(f'test logger.info')

2021-12-01 12:35:37,376 __main__ INFO test logger.info


In [8]:
_dir = os.getcwd()

In [9]:
logger.info(f'current working directory {_dir}')

2021-12-01 12:35:37,384 __main__ INFO current working directory /Users/joe.shea/excel_to_s3


In [10]:
def get_script_path():
    return os.path.dirname(os.path.realpath(sys.argv[0]))

In [11]:
_scriptpath = get_script_path()

In [12]:
logger.info(f'script path {_scriptpath}')

2021-12-01 12:35:37,398 __main__ INFO script path /Users/joe.shea/Library/Python/3.8/lib/python/site-packages


In [51]:
logger.info(f'Python version: {platform.python_version()}')

2021-12-01 13:25:57,165 __main__ INFO Python version: 3.8.9


In [52]:
logger.info(f'pandas version {pd.__version__}')

2021-12-01 13:26:03,517 __main__ INFO pandas version 1.3.4


## ***inputs***

In [13]:
_actor = 'joe.shea@revance.com'

In [14]:
inserted_by = _actor

In [15]:
updated_by = _actor

In [16]:
inserted_comment = _jiralink

In [17]:
_file = 'OLTP_20211124.xlsx'

In [18]:
_env_type = 'OLTP'

In [19]:
_env_dbtype = 'postgres'

In [20]:
logger.info(f'data governance file:{_file}')

2021-12-01 12:35:37,439 __main__ INFO data governance file:OLTP_20211124.xlsx


In [55]:
_filesplit  =_file.split('.')

In [56]:
_filesplit

['OLTP_20211124', 'xlsx']

In [21]:
xls = pd.ExcelFile(f'{_dir}/{_file}')
df = {sh.strip():xls.parse(sh) for i, sh in enumerate(xls.sheet_names)}
df.keys()

dict_keys(['gaia', 'kronos', 'p2pe', 'merchant', 'odf', 'chargeback', 'card', 'catalog', 'payment'])

In [22]:
assert len(df.keys()) == 9 ,"expecting dict_keys(['gaia', 'kronos', 'p2pe', 'merchant', 'odf', 'chargeback', 'card', 'catalog', 'payment'])"

In [23]:
type(df)

dict

In [24]:
type(df['gaia'])

pandas.core.frame.DataFrame

In [67]:
_currentIndex = 0

In [68]:
list(df.keys())[_currentIndex ]

'gaia'

In [69]:
_currentService = list(df.keys())[_currentIndex ]

In [25]:
df['gaia'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   host                      860 non-null    object 
 1   env                       860 non-null    object 
 2   service                   860 non-null    object 
 3   table_catalog             860 non-null    object 
 4   schemaname                860 non-null    object 
 5   table_name                860 non-null    object 
 6   column_name               860 non-null    object 
 7   data_type                 860 non-null    object 
 8   character_maximum_length  228 non-null    float64
 9   sensitive_data_type       39 non-null     object 
 10  comment(s)                42 non-null     object 
dtypes: float64(1), object(10)
memory usage: 74.0+ KB


In [26]:
df['gaia'].head()

,host,env,service,table_catalog,schemaname,table_name,column_name,data_type,character_maximum_length,sensitive_data_type,comment(s)
0,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,account,name,text,NaN,NaN,NaN
1,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,account,business_id,character varying,255.0,NaN,NaN
2,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,account,bank_name,character varying,255.0,UNCATEGORIZED,did not see any data
3,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,account,bank_account,character varying,255.0,UNCATEGORIZED,did not see any data
4,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,account,bank_routing,character varying,255.0,UNCATEGORIZED,did not see any data


In [27]:
df['gaia'].tail()

,host,env,service,table_catalog,schemaname,table_name,column_name,data_type,character_maximum_length,sensitive_data_type,comment(s)
855,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,userfield,canceled_at,timestamp without time zone,NaN,NaN,NaN
856,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,userfield,deleted_at,timestamp without time zone,NaN,NaN,NaN
857,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,userfield,encrypted_ref_id,character varying,NaN,NaN,NaN
858,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,userfield,status,smallint,NaN,NaN,NaN
859,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah,gaia,utah-gaia,public,userfield,id,bigint,NaN,NaN,NaN


In [28]:
list(df['gaia'].columns)

['host',
 'env',
 'service',
 'table_catalog',
 'schemaname',
 'table_name',
 'column_name',
 'data_type',
 'character_maximum_length',
 'sensitive_data_type',
 'comment(s)']

In [29]:
df['gaia'].drop('comment(s)', axis=1, inplace=True)

In [30]:
df['gaia'].drop('service', axis=1, inplace=True)

In [82]:
_none = None

<i>__change column names to match redshift control table__</i>

<i>SELECT id, "_env_dw", "_env_type", "_env_dbtype", "_host", "_database", "_schema", "_table", "_column", "_columnlength", sensitive_data_type, is_ready_for_encryption, is_encrypted, encryption_start, encryption_end, encrypted_row_count, code_message, is_encryption_confirmed, encryption_audit_by, encryption_audit_on, encryption_audit_comment, is_ready_for_transfer, inserted_by, inserted_on, inserted_comment, updated_by, updated_on, updated_comment, "_columntype", "_columnfeed"
FROM public.encrypt_sensitive_data_ctl;<i/>

In [31]:
df['gaia'].rename(columns={'host': '_host', 'env': '_env_dw','table_catalog': '_database','schemaname': '_schema','table_name': '_table','column_name': '_column','character_maximum_length': '_columnlength','sensitive_data_type': 'sensitive_data_type'}, inplace=True) 

In [32]:
df['gaia'].rename(columns={'data_type': '_columntype'}, inplace=True) 

In [33]:
df['gaia'].insert(len(df['gaia'].columns), 'inserted_by', inserted_by )

In [34]:
df['gaia'].insert(len(df['gaia'].columns), 'updated_by', updated_by )

In [35]:
df['gaia'].insert(len(df['gaia'].columns), 'inserted_comment',  inserted_comment)

In [36]:
df['gaia'].insert(len(df['gaia'].columns), '_env_type',  _env_type)

In [37]:
df['gaia'].insert(len(df['gaia'].columns), '_env_dbtype',  _env_dbtype)

In [83]:
df['gaia'].insert(len(df['gaia'].columns), 'id',  _none)

In [84]:
df['gaia'].insert(len(df['gaia'].columns), 'is_ready_for_encryption',  True)

In [85]:
df['gaia'].insert(len(df['gaia'].columns), 'is_encrypted',  False)

In [86]:
df['gaia'].insert(len(df['gaia'].columns), 'encryption_start',  _none)

In [87]:
df['gaia'].insert(len(df['gaia'].columns), 'encryption_end', _none )

In [88]:
df['gaia'].insert(len(df['gaia'].columns), 'encrypted_row_count',  _none)

In [89]:
df['gaia'].insert(len(df['gaia'].columns), 'code_message', _none )

In [90]:
df['gaia'].insert(len(df['gaia'].columns), 'is_encryption_confirmed',False  )

In [91]:
df['gaia'].insert(len(df['gaia'].columns), 'encryption_audit_by', _none )

In [92]:
df['gaia'].insert(len(df['gaia'].columns), 'encryption_audit_on', _none )

In [93]:
df['gaia'].insert(len(df['gaia'].columns), 'encryption_audit_comment', _none )

In [108]:
df['gaia'].insert(len(df['gaia'].columns), 'is_ready_for_transfer', False )

In [115]:
df['gaia'].insert(len(df['gaia'].columns), 'inserted_on', _none )

In [96]:
df['gaia'].insert(len(df['gaia'].columns), 'updated_on', _none )

In [97]:
df['gaia'].insert(len(df['gaia'].columns), 'updated_comment', _none )

In [98]:
df['gaia'].insert(len(df['gaia'].columns), '_columnfeed',  _none)

<i> __reorder columns__ </i>

In [116]:
df['gaia'] = df['gaia'][[ 'id','_env_dw','_env_type','_env_dbtype','_host', '_database','_schema','_table','_column','_columnlength','sensitive_data_type','is_ready_for_encryption','is_encrypted','encryption_start', 'encryption_end','encrypted_row_count','code_message','is_encryption_confirmed','encryption_audit_by','encryption_audit_on','encryption_audit_comment','is_ready_for_transfer','inserted_by','inserted_on','inserted_comment','updated_by','updated_on', 'updated_comment','_columntype' ,'_columnfeed']]

In [117]:
list(df['gaia'].columns)

['id',
 '_env_dw',
 '_env_type',
 '_env_dbtype',
 '_host',
 '_database',
 '_schema',
 '_table',
 '_column',
 '_columnlength',
 'sensitive_data_type',
 'is_ready_for_encryption',
 'is_encrypted',
 'encryption_start',
 'encryption_end',
 'encrypted_row_count',
 'code_message',
 'is_encryption_confirmed',
 'encryption_audit_by',
 'encryption_audit_on',
 'encryption_audit_comment',
 'is_ready_for_transfer',
 'inserted_by',
 'inserted_on',
 'inserted_comment',
 'updated_by',
 'updated_on',
 'updated_comment',
 '_columntype',
 '_columnfeed']

In [41]:
df['gaia'] = df['gaia'][df['gaia']['sensitive_data_type'].notnull()]

In [118]:
df['gaia'].head()

,id,_env_dw,_env_type,_env_dbtype,_host,_database,_schema,_table,_column,_columnlength,...,encryption_audit_comment,is_ready_for_transfer,inserted_by,inserted_on,inserted_comment,updated_by,updated_on,updated_comment,_columntype,_columnfeed
2,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,bank_name,255.0,...,None,False,joe.shea@revance.com,None,https://hintmd.atlassian.net/browse/OTECH-2685,joe.shea@revance.com,None,None,character varying,None
3,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,bank_account,255.0,...,None,False,joe.shea@revance.com,None,https://hintmd.atlassian.net/browse/OTECH-2685,joe.shea@revance.com,None,None,character varying,None
4,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,bank_routing,255.0,...,None,False,joe.shea@revance.com,None,https://hintmd.atlassian.net/browse/OTECH-2685,joe.shea@revance.com,None,None,character varying,None
5,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,account,identifier,255.0,...,None,False,joe.shea@revance.com,None,https://hintmd.atlassian.net/browse/OTECH-2685,joe.shea@revance.com,None,None,character varying,None
24,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,address,name,NaN,...,None,False,joe.shea@revance.com,None,https://hintmd.atlassian.net/browse/OTECH-2685,joe.shea@revance.com,None,None,text,None


In [143]:
df['gaia'].tail()

,id,_env_dw,_env_type,_env_dbtype,_host,_database,_schema,_table,_column,_columnlength,...,encryption_audit_comment,is_ready_for_transfer,inserted_by,inserted_on,inserted_comment,updated_by,updated_on,updated_comment,_columntype,_columnfeed
617,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,product,name,0,...,None,False,joe.shea@revance.com,None,https://hintmd.atlassian.net/browse/OTECH-2685,joe.shea@revance.com,None,None,text,None
633,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,provider,name,0,...,None,False,joe.shea@revance.com,None,https://hintmd.atlassian.net/browse/OTECH-2685,joe.shea@revance.com,None,None,text,None
685,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,refund,reason,0,...,None,False,joe.shea@revance.com,None,https://hintmd.atlassian.net/browse/OTECH-2685,joe.shea@revance.com,None,None,text,None
717,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,refund_transaction,last4,10,...,None,False,joe.shea@revance.com,None,https://hintmd.atlassian.net/browse/OTECH-2685,joe.shea@revance.com,None,None,character varying,None
785,None,utah,OLTP,postgres,hintdb-atlas.cluster-ro-cxfeateo6sft.us-east-1...,utah-gaia,public,subscription,name,0,...,None,False,joe.shea@revance.com,None,https://hintmd.atlassian.net/browse/OTECH-2685,joe.shea@revance.com,None,None,text,None


In [44]:
df['gaia'] = df['gaia'][~df['gaia']['sensitive_data_type'].isin(['all ready encrypted'])]

In [130]:
df['gaia'].value_counts(subset=['sensitive_data_type'],dropna=False) 

sensitive_data_type
UNCATEGORIZED          23
PII                    11
PCI                     4
dtype: int64

In [160]:
df['gaia'].value_counts(subset=['_columnlength'],dropna=False) 

_columnlength
0                34
4                 2
8                 1
10                1
dtype: int64

In [156]:
mask_columnlength = df['gaia']['_columnlength'] < 0

In [157]:
mask_columnlength_name = '_columnlength'

In [159]:
df['gaia'].loc[mask_columnlength,mask_columnlength_name] = 0

In [134]:
df['gaia']['_columnlength']=df['gaia']['_columnlength'].fillna(0)

In [138]:
df['gaia']['_columnlength']=df['gaia']['_columnlength'].astype(np.int8)

In [162]:
df['gaia'].drop('id', axis=1, inplace=True)

In [163]:
df['gaia'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 2 to 785
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   _env_dw                   38 non-null     object
 1   _env_type                 38 non-null     object
 2   _env_dbtype               38 non-null     object
 3   _host                     38 non-null     object
 4   _database                 38 non-null     object
 5   _schema                   38 non-null     object
 6   _table                    38 non-null     object
 7   _column                   38 non-null     object
 8   _columnlength             38 non-null     int8  
 9   sensitive_data_type       38 non-null     object
 10  is_ready_for_encryption   38 non-null     bool  
 11  is_encrypted              38 non-null     bool  
 12  encryption_start          0 non-null      object
 13  encryption_end            0 non-null      object
 14  encrypted_row_count       0

In [146]:
_compressionOut ='None'

In [147]:
#_fileout_suffix = 'parquet'

In [151]:
_fileout_suffix = 'csv'

In [152]:
_fileout = _filesplit[0]+'_'+_currentService+'.'+_compressionOut+'.'+_fileout_suffix

In [153]:
logger.info(f'file out for s3:{_fileout}')

2021-12-04 15:38:01,056 __main__ INFO file out for s3:OLTP_20211124_gaia.None.csv


In [150]:
#df['gaia'].to_parquet(f'{_fileout}', compression=_compressionOut, index=False)

header = False

In [164]:
df['gaia'].to_csv(f'{_fileout}',index=False)